# Simplified Analysis and Visualization Script
Only retains the figures and corresponding tests needed in the article and supplementary materials  
Please first rename the corresponding 'allresult_processed.csv' generated by different parameters simulation and put it in the current directory

## Load necessary packages
```r
library(tidyverse)
library(ggpubr)
library(dplyr)
```

## Define Wilcoxon test function
```r
wilcox_test <- function(x, y, alternative) {
  test <- wilcox.test(x, y, alternative = alternative)
  return(test$p.value)
}
```

## WSLS vs Baseline

### Data loading
```r
baseline_data_wsls <- read_csv("baseline_1500s_allresult_processed.csv")
WSLS_data <- read_csv("WSLS_1500s_allresult_processed.csv")
baseline_data_wsls$group <- "Baseline"
WSLS_data$group <- "WSLS"
all_data_wsls <- bind_rows(baseline_data_wsls, WSLS_data) %>%
  mutate(Subject = paste0(group, "_", Subject)) %>%
  arrange(Subject, Round, Phase)
```

### Figure 2a: Histogram of consecutive schema selections
```r
streak_data <- all_data_wsls %>%
  group_by(Subject, Round) %>%
  summarise(Schema_Phase1 = first(Schema), Schema_Phase2 = last(Schema), group = first(group)) %>%
  arrange(Subject, Round) %>%
  group_by(Subject) %>%
  mutate(
    is_continuous = (Schema_Phase1 == lag(Schema_Phase1, default = NA) |
                       Schema_Phase1 == lag(Schema_Phase2, default = NA) |
                       Schema_Phase2 == lag(Schema_Phase1, default = NA) |
                       Schema_Phase2 == lag(Schema_Phase2, default = NA)),
    streak_start = !is_continuous | is.na(is_continuous),
    streak_id = cumsum(streak_start)
  ) %>%
  group_by(Subject, streak_id) %>%
  summarise(streak_length = n(), group = first(group)) %>%
  ungroup()
streak_freq <- streak_data %>%
  group_by(group, streak_length) %>%
  summarise(count = n(), .groups = "drop") %>%
  complete(group, streak_length = 1:max(streak_length), fill = list(count = 0))
baseline_streaks <- streak_data %>% filter(group == "Baseline") %>% pull(streak_length)
wsls_streaks <- streak_data %>% filter(group == "WSLS") %>% pull(streak_length)
ks_result <- ks.test(baseline_streaks, wsls_streaks)
p_value <- ks_result$p.value
p_label <- ifelse(p_value < 0.001, "p < 0.001", sprintf("p = %.3f", p_value))
p1 <- ggplot(streak_freq, aes(x = streak_length, y = count, fill = group)) +
  geom_bar(stat = "identity", position = "dodge") +
  scale_fill_manual(values = c("Baseline" = "#1F77B4", "WSLS" = "#FF7F0E")) +
  labs(title = "Sequential Schema Selection in WSLS", x = "Consecutive Number", y = "Sequence Number") +
  theme_minimal(base_size = 12) +
  theme(legend.position = "top", plot.title = element_text(hjust = 0.5, size = 14, face = "bold")) +
  annotate("text", x = Inf, y = Inf, label = p_label, hjust = 1.1, vjust = 1.1, size = 4)
```

### WSLS classification and comparison with Baseline (for Supplementary Figure 1)
```r
wsls_classified <- WSLS_data %>%
  arrange(Subject, Round, Phase) %>%
  group_by(Subject) %>%
  mutate(
    prev_Schema1 = lag(Schema, n = 2, default = NA),
    prev_Schema2 = lag(Schema, n = 1, default = NA),
    prev_AC1 = lag(AC, n = 2, default = NA),
    prev_AC2 = lag(AC, n = 1, default = NA)
  ) %>%
  mutate(
    same_schema = (Schema == prev_Schema1 | Schema == prev_Schema2),
    prev_AC = case_when(
      Schema == prev_Schema1 & Schema != prev_Schema2 ~ prev_AC1,
      Schema == prev_Schema2 & Schema != prev_Schema1 ~ prev_AC2,
      Schema == prev_Schema1 & Schema == prev_Schema2 ~ pmax(prev_AC1, prev_AC2, na.rm = TRUE),
      TRUE ~ NA_real_
    ),
    prev_AC_no_match = ifelse(same_schema, NA_real_, prev_AC2),
    condition = case_when(
      is.na(prev_Schema1) | is.na(prev_Schema2) ~ NA_character_,
      same_schema & prev_AC == 1 ~ "Same, AC=1",
      same_schema & prev_AC != 1 ~ "Same, AC!=1",
      !same_schema & prev_AC_no_match == 1 ~ "Diff, AC=1",
      !same_schema & prev_AC_no_match != 1 ~ "Diff, AC!=1"
    )
  ) %>%
  ungroup()
```

### Combine WSLS and Baseline data
```r
analysis_data <- bind_rows(
  wsls_classified %>% select(condition, Schema_RT, Schema_OB, Schema_AS) %>% filter(!is.na(condition)),
  baseline_data_wsls %>% mutate(condition = "Baseline") %>% select(condition, Schema_RT, Schema_OB, Schema_AS)
) %>%
  filter(!is.na(Schema_RT), !is.na(Schema_OB), !is.na(Schema_AS))
```

### Supplementary Figure 1: Violin plots of behavioral metrics
```r
long_data <- analysis_data %>%
  pivot_longer(
    cols = c(Schema_RT, Schema_OB, Schema_AS),
    names_to = "metric",
    values_to = "value"
  ) %>%
  mutate(
    metric = factor(metric, 
                    levels = c("Schema_RT", "Schema_OB", "Schema_AS"),
                    labels = c("Reaction Time (s)", "Observation Counts", "Attention Shifts"))
  )

combined_violin <- ggplot(long_data, aes(x = condition, y = value, fill = condition)) +
  geom_violin(trim = TRUE) +
  scale_fill_manual(values = c("Same, AC=1" = "#FF7F0E", "Same, AC!=1" = "#D62728", 
                               "Diff, AC=1" = "#2CA02C", "Diff, AC!=1" = "#9467BD", 
                               "Baseline" = "#1F77B4")) +
  facet_wrap(~ metric, scales = "free_y", ncol = 1) +
  labs(title = "Schema Performance Metrics", x = "Condition", y = NULL) +
  theme_minimal(base_size = 12) +
  theme(
    legend.position = "top",
    plot.title = element_text(hjust = 0.5, size = 14, face = "bold"),
    strip.text = element_text(size = 12, face = "bold"),
    axis.text.x = element_text(angle = 45, hjust = 1)
  )
```

### Kruskal-Wallis test
```r
kw_tests <- long_data %>%
  group_by(metric) %>%
  summarise(
    p_value = kruskal.test(value ~ condition)$p.value,
    .groups = "drop"
  ) %>%
  mutate(
    p_label = ifelse(p_value < 0.001, "p < 0.001", sprintf("p = %.3f", p_value))
  )
```

### Figure 2b: Bubble plot of schema matching accuracy
```r
ac_freq_wsls <- all_data_wsls %>%
  group_by(group, Round, AC) %>%
  summarise(count = n(), .groups = "drop") %>%
  filter(!is.na(AC))
p6 <- ggplot(ac_freq_wsls, aes(x = Round, y = AC, size = count, color = group)) +
  geom_point(position = position_dodge(width = 0.8), alpha = 0.7) +
  scale_size_continuous(range = c(2, 10)) +
  scale_color_manual(values = c("Baseline" = "#1F77B4", "WSLS" = "#FF7F0E")) +
  labs(title = "Accuracy Distribution (WSLS vs Baseline)", x = "Round", y = "Schema Accuracy", size = "Number", color = "Group") +
  theme_minimal(base_size = 12) +
  theme(legend.position = "top", plot.title = element_text(hjust = 0.5, size = 14, face = "bold"))
```

## Feedback vs Baseline

### Data loading
```r
baseline_data_feedback <- read_csv("baseline_1500s_allresult_processed.csv")
feedback_data <- read_csv("feedback_allresult_processed.csv")
excitement_data <- read_csv("excitement_allresult_processed.csv")
depression_data <- read_csv("depression_allresult_processed.csv")
baseline_data_feedback$group <- "Baseline"
feedback_data$group <- "Feedback"
excitement_data$group <- "Excitement"
depression_data$group <- "Depression"
all_data_feedback <- bind_rows(baseline_data_feedback, feedback_data, excitement_data, depression_data) %>%
  mutate(Subject = paste0(group, "_", Subject)) %>%
  arrange(Subject, Round, Phase)
```

### Figure 2d: Performance under different emotional states
```r
performance_by_round_feedback <- all_data_feedback %>%
  group_by(group, Round) %>%
  summarise(
    mean_performance = mean(performance, na.rm = TRUE),
    se_performance = sd(performance, na.rm = TRUE) / sqrt(n())
  )
p7 <- ggplot(performance_by_round_feedback, aes(x = Round, y = mean_performance, color = group)) +
  geom_line(size = 1) +
  geom_ribbon(aes(ymin = mean_performance - se_performance, ymax = mean_performance + se_performance), alpha = 0.08, linetype = 0) +
  scale_color_manual(values = c("Baseline" = "#1F77B4", "Feedback" = "#FF7F0E", "Depression" = "#2CA02C", "Excitement" = "#D62728")) +
  labs(title = "Average Performance For Each Round", x = "Round", y = "Average Performance Score") +
  theme_minimal(base_size = 12) +
  theme(legend.position = "top", plot.title = element_text(hjust = 0.5, size = 14, face = "bold"))
```

### Figure 2e: Violin plot of reaction times under different emotional states
```r
p8 <- ggplot(all_data_feedback, aes(x = group, y = Schema_RT, fill = group)) +
  geom_violin(trim = TRUE) +
  scale_fill_manual(values = c("Baseline" = "#1F77B4", "Feedback" = "#FF7F0E", "Depression" = "#2CA02C", "Excitement" = "#D62728")) +
  labs(title = "Reaction Times under Different Emotional Feedback", x = "Emotion Group", y = "Schema Reaction Time (s)") +
  theme_minimal(base_size = 12) +
  theme(legend.position = "none", plot.title = element_text(hjust = 0.5, size = 14, face = "bold"))

# Kruskal-Wallis test for reaction times
kruskal_result_rt_feedback <- kruskal.test(Schema_RT ~ group, data = all_data_feedback)
p_value_rt_feedback <- kruskal_result_rt_feedback$p.value
p_label_rt_feedback <- ifelse(p_value_rt_feedback < 0.001, "p < 0.001", sprintf("p = %.3f", p_value_rt_feedback))
p8 <- p8 + annotate("text", x = Inf, y = Inf, label = p_label_rt_feedback, hjust = 1.1, vjust = 1.1, size = 4)
```

## Payoff vs Baseline

### Data loading
```r
baseline_1500s_data_payoff <- read_csv("baseline_1500s_allresult_processed.csv")
payoff_1500s_data <- read_csv("payoff_1500s_allresult_processed.csv")

baseline_1500s_data_payoff <- baseline_1500s_data_payoff %>%
  select(-Schema, -schema_payoff)
payoff_1500s_data <- payoff_1500s_data %>%
  select(-Schema, -schema_payoff)

baseline_1500s_data_payoff$group <- "Baseline_1500s"
payoff_1500s_data$group <- "Payoff_1500s"

data_1500s_payoff <- bind_rows(baseline_1500s_data_payoff, payoff_1500s_data) %>%
  mutate(Subject = paste0(group, "_", Subject)) %>%
  arrange(Subject, Round, Phase)
```

### Supplementary Figure 2: Bubble plot of accuracy distribution for Payoff vs Baseline
```r
ac_freq_payoff <- data_1500s_payoff %>%
  filter(Round <= 5) %>%
  group_by(group, Round, AC) %>%
  summarise(count = n(), .groups = "drop") %>%
  filter(!is.na(AC))
p13 <- ggplot(ac_freq_payoff, aes(x = Round, y = AC, size = count, color = group)) +
  geom_point(position = position_dodge(width = 0.8), alpha = 0.7) +
  scale_size_continuous(range = c(2, 10)) +
  scale_color_manual(values = c("Baseline_1500s" = "#1F77B4", "Payoff_1500s" = "#FF7F0E")) +
  labs(title = "Accuracy Distribution ('High-Payoff' vs Baseline)", x = "Round", y = "Accuracy", size = "Number", color = "Group") +
  theme_minimal(base_size = 12) +
  theme(legend.position = "top", plot.title = element_text(hjust = 0.5, size = 14, face = "bold"))
```

## WSLS vs Payoff

### 1500s Data loading
```r
baseline_1500s_data_wsls_payoff <- read_csv("baseline_1500s_allresult_processed.csv")
WSLS_1500s_data <- read_csv("WSLS_1500s_allresult_processed.csv")
payoff_1500s_data_wsls <- read_csv("payoff_1500s_allresult_processed.csv")
integration_1500s_data <- read_csv("integration_1500s_allresult_processed.csv")

baseline_1500s_data_wsls_payoff <- baseline_1500s_data_wsls_payoff %>%
  select(-Schema, -schema_payoff)
WSLS_1500s_data <- WSLS_1500s_data %>%
  select(-Schema, -schema_payoff)
payoff_1500s_data_wsls <- payoff_1500s_data_wsls %>%
  select(-Schema, -schema_payoff)
integration_1500s_data <- integration_1500s_data %>%
  select(-Schema, -schema_payoff)

baseline_1500s_data_wsls_payoff$group <- "Baseline_1500s"
WSLS_1500s_data$group <- "WSLS_1500s"
payoff_1500s_data_wsls$group <- "Payoff_1500s"
integration_1500s_data$group <- "Integration_1500s"

all_data_1500s_wsls_payoff <- bind_rows(baseline_1500s_data_wsls_payoff, WSLS_1500s_data, payoff_1500s_data_wsls, integration_1500s_data) %>%
  mutate(Subject = paste0(group, "_", Subject)) %>%
  arrange(Subject, Round, Phase)
```

### Figure 2c: Performance of different adaptive strategies in 1500s
```r
performance_by_round_1500s_wsls_payoff <- all_data_1500s_wsls_payoff %>%
  group_by(group, Round) %>%
  summarise(
    mean_performance = mean(performance, na.rm = TRUE),
    se_performance = sd(performance, na.rm = TRUE) / sqrt(n())
  )
p14 <- ggplot(performance_by_round_1500s_wsls_payoff, aes(x = Round, y = mean_performance, color = group)) +
  geom_line(size = 1) +
  geom_ribbon(aes(ymin = mean_performance - se_performance, ymax = mean_performance + se_performance), alpha = 0.08, linetype = 0) +
  scale_color_manual(values = c("Baseline_1500s" = "#1F77B4", "WSLS_1500s" = "#FF7F0E", "Payoff_1500s" = "#2CA02C", "Integration_1500s" = "#D62728")) +
  labs(title = "Performance of Different Adaptive Strategies in 1500s", x = "Round", y = "Average Performance Score") +
  theme_minimal(base_size = 12) +
  theme(legend.position = "top", plot.title = element_text(hjust = 0.5, size = 14, face = "bold"))
```

### 2500s Data loading
```r
baseline_2500s_data_wsls_payoff <- read_csv("baseline_2500s_allresult_processed.csv")
WSLS_2500s_data <- read_csv("WSLS_2500s_allresult_processed.csv")
payoff_2500s_data_wsls <- read_csv("payoff_2500s_allresult_processed.csv")
integration_2500s_data <- read_csv("integration_2500s_allresult_processed.csv")

baseline_2500s_data_wsls_payoff <- baseline_2500s_data_wsls_payoff %>%
  select(-Schema, -schema_payoff)
WSLS_2500s_data <- WSLS_2500s_data %>%
  select(-Schema, -schema_payoff)
payoff_2500s_data_wsls <- payoff_2500s_data_wsls %>%
  select(-Schema, -schema_payoff)
integration_2500s_data <- integration_2500s_data %>%
  select(-Schema, -schema_payoff)

baseline_2500s_data_wsls_payoff$group <- "Baseline_2500s"
WSLS_2500s_data$group <- "WSLS_2500s"
payoff_2500s_data_wsls$group <- "Payoff_2500s"
integration_2500s_data$group <- "Integration_2500s"

all_data_2500s_wsls_payoff <- bind_rows(baseline_2500s_data_wsls_payoff, WSLS_2500s_data, payoff_2500s_data_wsls, integration_2500s_data) %>%
  mutate(Subject = paste0(group, "_", Subject)) %>%
  arrange(Subject, Round, Phase)
```

### Figure 2f: Performance of different adaptive strategies in 2500s
```r
performance_by_round_2500s_wsls_payoff <- all_data_2500s_wsls_payoff %>%
  group_by(group, Round) %>%
  summarise(
    mean_performance = mean(performance, na.rm = TRUE),
    se_performance = sd(performance, na.rm = TRUE) / sqrt(n())
  )
p15 <- ggplot(performance_by_round_2500s_wsls_payoff, aes(x = Round, y = mean_performance, color = group)) +
  geom_line(size = 1) +
  geom_ribbon(aes(ymin = mean_performance - se_performance, ymax = mean_performance + se_performance), alpha = 0.08, linetype = 0) +
  scale_color_manual(values = c("Baseline_2500s" = "#1F77B4", "WSLS_2500s" = "#FF7F0E", "Payoff_2500s" = "#2CA02C", "Integration_2500s" = "#D62728")) +
  labs(title = "Performance of Different Adaptive Strategies in 2500s", x = "Round", y = "Average Performance Score") +
  theme_minimal(base_size = 12) +
  theme(legend.position = "top", plot.title = element_text(hjust = 0.5, size = 14, face = "bold"))
```

## Save Figures

### Save figures for WSLS vs Baseline
```r
ggsave("fig2a_sequential_schema.png", p1, width = 5, height = 7, dpi = 300)
ggsave("supplementary_fig1_behavioral_metrics.png", combined_violin, width = 8, height = 12, dpi = 300)
ggsave("fig2b_accuracy_distribution.png", p6, width = 8, height = 7, dpi = 300)
```

### Save figures for Feedback vs Baseline
```r
ggsave("fig2d_performance_emotional_states.png", p7, width = 8, height = 7, dpi = 300)
ggsave("fig2e_reaction_times_emotional_states.png", p8, width = 5, height = 7, dpi = 300)
```

### Save figures for Payoff vs Baseline
```r
ggsave("supplementary_fig2_accuracy_distribution_payoff.png", p13, width = 8, height = 7, dpi = 300)
```

### Save figures for WSLS vs Payoff
```r
ggsave("fig2c_performance_1500s.png", p14, width = 8, height = 7, dpi = 300)
ggsave("fig2f_performance_2500s.png", p15, width = 8, height = 7, dpi = 300)
```